In [1]:
include("../src/RadialBasisFunction.jl")

using .RadialBasisFunction
using LinearAlgebra
using JLD2

## Defining the problem

Points

In [2]:
a, b = 0, 2π
N = 40
X = range(a,b, length=N)
boundary_idx = [1, N]
X_boundary = X[boundary_idx]
internal_idx = 2:(N-1)
X_internal = X[internal_idx]

0.16110731556870733:0.16110731556870733:6.122077991610879

Generic steady-state (heat) equation  

$$ \Delta u = q $$

In [3]:
u(x) = sin.(x)   # Not known in advance
q(x) = -sin.(x)  # Known in advance (param)

boundary_conditions = u(X_boundary)

2-element Vector{Float64}:
  0.0
 -2.4492935982947064e-16

The RBF interpolator centered on $\textit{i-th}$ point of a stencil  

$$ u(x) \approx u^{h}(x) \coloneqq \sum_{i=1}^{n} \alpha_{i} \varphi(x, x_i) + \sum_{j=1}^{m} \beta_{j} p_j(x)

In [4]:
φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)

ddφ (generic function with 1 method)

Stencil definition

In [5]:
n_points_in_stencil = 7

7

## Solution

Solving field equation using RBF approximation

In [6]:
C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idx, boundary_idx, boundary_conditions, q)

38-element Vector{Float64}:
  0.16230630440607596
  0.3184441606794323
  0.46639819882924805
  0.6024453236002578
  0.7229552007955176
  0.8247685909891668
  0.9052437503888441
  0.9622961535265828
  0.9944469507504687
  1.000861661907953
  ⋮
 -0.9944469507504649
 -0.96229615352658
 -0.9052437503888425
 -0.8247685909891656
 -0.7229552007955165
 -0.602445323600257
 -0.46639819882924743
 -0.31844416067943265
 -0.16230630440607627

## Performances

In [7]:
rel_error_on_u = norm(u_approximated - u(X_internal)) / norm(u(X_internal))

0.002042852922791229

## Store results

In [13]:
# const resulting_datas_dir = "../results/datas/"

# variable_to_store = Dict("C" => C, "q" => q(X), "f" => C[internal_idx, internal_idx] * boundary_conditions,
#                          "X" => X, "X_internal" => X_internal, "X_boundary" => X_boundary,
#                          "")
# save(resulting_datas_dir * "vanilla_RBF_problem.jld2", "u_num", u_approximated)
# save(resulting_datas_dir * "vanilla_RBF_solution.jld2", "u_num", u_approximated)